## Import Packages

In [10]:
import csv
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

## Extract Data

In [11]:
def findCommunities(filename):
    communities = []
    with open(filename, 'r') as csvfile:
        filereader = csv.reader(csvfile)
        j = 0
        for i in filereader:
            if j > 0:
                if i[1] not in communities:
                    communities.append(i[1])
            j+=1
    return communities
def extractData(filename,communities,addstate = False,addcommunity = False):
    features = []
    xPositive=[]
    yPositive=[]
    xNegative=[]
    yNegative =[]
    with open(filename, 'r') as csvfile:
        filereader = csv.reader(csvfile)
        j = 0
        for i in filereader:
            if j > 0:
                data = i[5:len(i)-1]
                while '?' in data:
                    data[data.index('?')] = 0
                if addstate:
                    state = [0.0]*56
                    state[int(i[0])-1] = 1.0
                    data = data + state
                    features = features + ["state"+str(i) for i in range(56)]
                if addcommunity:
                    tmp = [0.0]*len(communities)
                    tmp[communities.index(i[1])] = 1.0
                    data = data + tmp
                    features = features + communities
                vect = np.array(data)
                #vect = np.array(i[3:len(i)-1])
                #x.append(vect.astype(np.float))
                if float(i[-1]) > 0.1:
                    xPositive.append(vect.astype(np.float))
                    yPositive.append(1)
                else:
                    xNegative.append(vect.astype(np.float))
                    yNegative.append(0)
            else:
                features = i[5:len(i)-1]
            j+=1
        """yPositive = np.array(yPositive)
        xPositive = np.array(xPositive)
        yNegative = np.array(yNegative)
        xNegative = np.array(xNegative)"""
        features = np.array(features)
        indxP = int(len(xPositive)*.6)
        indxN = int(len(xNegative)*.6)
        xTrain =np.array(xPositive[:indxP] + xNegative[:indxN])
        yTrain = np.array(yPositive[:indxP] + yNegative[:indxN])
        xTest = np.array(xPositive[indxP:] + xNegative[indxN:])
        yTest = np.array(yPositive[indxP:] + yNegative[indxN:])
        
        
            
            
        return xTrain,yTrain,xTest,yTest,features

## Calculate Label Percentage

In [12]:
def calcualtePercentage(labels):
    n = len(labels)
    sumTrue = sum(labels)*1.0
    sumFalse = (n - sumTrue)*1.0
    return sumTrue/n,sumFalse/n

## Create Model

In [13]:
def clfSVC(kernel = 'rbf'):
    return SVC(C=1.0, kernel= kernel, degree=3, gamma='auto', coef0=0.0,
               shrinking=True, probability=False, tol=0.001, cache_size=200, 
               class_weight=None, verbose=False, max_iter=-1,decision_function_shape=None, random_state=None)

## Evaluation

In [14]:
def confusionMatrix(true,predicted,labels = [1,0]):
    return confusion_matrix(true,predicted,labels)

def find_confusion_matrix(actual,predicted,clabels):
    cm= []
    for i in clabels:
        tmp =[0]*len(clabels)
        
        for j in range(len(actual)):
            if actual[j]== i and actual[j] == predicted[j]:
                tmp[clabels.index(i)] += 1
            elif actual[j]== i and actual[j] != predicted[j]:
                tmp[clabels.index(predicted[j])] += 1
        cm.append(tmp)
    return np.array(cm)
def find_accuracy(matrix):
    return np.trace(matrix)*1.0/np.sum(matrix)
def find_precision(matrix):
    pres = []
    x = np.sum(matrix,axis=0)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                pres.append(matrix[i][j]*1.0/x[i])
    return pres
def find_recall(matrix):
    rec = []
    x = np.sum(matrix,axis=1)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                rec.append(matrix[i][j]*1.0/x[i])
    return rec
def evaluation(acutal,predicted,clabels=[1,0]):
    confmatrix = find_confusion_matrix(acutal,predicted,clabels)
    print ("Confusion Matrix")
    print (confmatrix)
    accuracy = find_accuracy(confmatrix)
    print ("Accuracy", accuracy)
    precision = find_precision(confmatrix)
    print ("Precision", precision)
    recall = find_recall(confmatrix)
    print ("Recall", recall)

## Cross Validation

In [15]:
def do_cross_validation(X, y,clf, n_folds=5):
    cv = KFold(len(y), n_folds, shuffle=True)
    accuracies = []
    i = 1
    for train_ind, test_ind in cv: 
        clf.fit(X[train_ind], y[train_ind])
        predictions = clf.predict(X[test_ind])
        print ("Fold %d Evaluation" %i)
        i +=1
        evaluation(y[test_ind],predictions)
        print ("\n")
        accuracies.append(accuracy_score(y[test_ind], predictions))
    avg = np.mean(accuracies)
    print ("The average Accuracy is ", avg)

### Read Data

In [16]:
filename = 'C:\\Users\\ar1\\Documents\ML\\Project\\Crime Prediction Data(1)\\Crime Prediction Data\\communities-crime-full.csv'
distintCommunities = findCommunities(filename)
XTrain,YTrain,XTest,YTest,Features = extractData(filename, distintCommunities)

### Percentage of Labels

In [17]:
print ("Train Percentage",calcualtePercentage(YTrain))
print ("Test Percentage",calcualtePercentage(YTest))

Train Percentage (0.62761506276150625, 0.3723849372384937)
Test Percentage (0.62703379224030042, 0.37296620775969963)


### Default SVC Model

In [18]:
model= clfSVC()
model.fit(XTrain,YTrain)
predictedY=model.predict(XTest)
evaluation(YTest,predictedY)

Confusion Matrix
[[426  75]
 [ 73 225]]
Accuracy 0.814768460576
Precision [0.85370741482965928, 0.75]
Recall [0.85029940119760483, 0.75503355704697983]


### 10 Fold Cross Validation on Default SVC Model

In [19]:
do_cross_validation(XTrain,YTrain,model,10)

Fold 1 Evaluation
Confusion Matrix
[[68 10]
 [10 32]]
Accuracy 0.833333333333
Precision [0.87179487179487181, 0.76190476190476186]
Recall [0.87179487179487181, 0.76190476190476186]


Fold 2 Evaluation
Confusion Matrix
[[58  9]
 [19 34]]
Accuracy 0.766666666667
Precision [0.75324675324675328, 0.79069767441860461]
Recall [0.86567164179104472, 0.64150943396226412]


Fold 3 Evaluation
Confusion Matrix
[[72  8]
 [12 28]]
Accuracy 0.833333333333
Precision [0.8571428571428571, 0.77777777777777779]
Recall [0.90000000000000002, 0.69999999999999996]


Fold 4 Evaluation
Confusion Matrix
[[67 17]
 [ 8 28]]
Accuracy 0.791666666667
Precision [0.89333333333333331, 0.62222222222222223]
Recall [0.79761904761904767, 0.77777777777777779]


Fold 5 Evaluation
Confusion Matrix
[[68  4]
 [17 31]]
Accuracy 0.825
Precision [0.80000000000000004, 0.88571428571428568]
Recall [0.94444444444444442, 0.64583333333333337]


Fold 6 Evaluation
Confusion Matrix
[[66  6]
 [14 33]]
Accuracy 0.831932773109
Precision [0.8249

### Evaluation for Different Kernels

In [21]:
kernel = ['linear', 'rbf', 'sigmoid',]
modelList =[]
for i in kernel:
    model= clfSVC(kernel = i)
    model.fit(XTrain,YTrain)
    modelList.append(model)
    predictedY=model.predict(XTest)
    print("\n"+i)
    evaluation(YTest,predictedY)
    


linear
Confusion Matrix
[[423  78]
 [ 60 238]]
Accuracy 0.827284105131
Precision [0.87577639751552794, 0.75316455696202533]
Recall [0.84431137724550898, 0.79865771812080533]

rbf
Confusion Matrix
[[426  75]
 [ 73 225]]
Accuracy 0.814768460576
Precision [0.85370741482965928, 0.75]
Recall [0.85029940119760483, 0.75503355704697983]

sigmoid
Confusion Matrix
[[422  79]
 [ 84 214]]
Accuracy 0.795994993742
Precision [0.83399209486166004, 0.7303754266211604]
Recall [0.8423153692614771, 0.71812080536912748]


### Top 10 important features in Linear model

In [22]:
coef = modelList[0].coef_
topindx = np.argsort(coef)[0][::-1][:10]
print (Features[topindx])


['racepctblack' 'MedRentPctHousInc' 'LemasGangUnitDeploy'
 'PersPerRentOccHous' 'PctEmploy' 'PersPerOccupHous' 'PctNotSpeakEnglWell'
 'population' 'agePct12t21' 'PctImmigRecent']
